## Install packages

In [1]:
#! pip install vllm
!pip install vllm==0.3.3
!pip install flash-attn==2.5.6

from vllm import LLM
from vllm import SamplingParams
import pandas as pd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 MB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.1/65.1 MB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 MB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.6/83.6 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 MB 19.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cach

## Load model

In [ ]:
try:
    del llm
except:
    print("first model loading")

# llm = LLM(model='mistralai/Mistral-7B-Instruct-v0.1')
# llm = LLM(model='TheBloke/Mistral-7B-Instruct-v0.2-AWQ')
llm = LLM(model='BioMistral/BioMistral-7B-DARE-AWQ-QGS128-W4-GEMM')

first model loading


config.json:   0%|          | 0.00/887 [00:00<?, ?B/s]

WARNING 04-15 23:19:49 config.py:193] awq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 04-15 23:19:49 llm_engine.py:87] Initializing an LLM engine with config: model='BioMistral/BioMistral-7B-DARE-AWQ-QGS128-W4-GEMM', tokenizer='BioMistral/BioMistral-7B-DARE-AWQ-QGS128-W4-GEMM', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=awq, enforce_eager=False, kv_cache_dtype=auto, device_config=cuda, seed=0)


tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

INFO 04-15 23:19:54 weight_utils.py:163] Using model weights format ['*.safetensors']


model.safetensors:   0%|          | 0.00/4.15G [00:00<?, ?B/s]

## Import data from google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
proj_path = "/content/drive/My Drive/ubc/COLX531/"
data_set = "PLOS" #"eLife" #

data_type = "test" # "val" #"val_mini" #
model_name = "biomistral_7b_awq"
method = "abstract_keyword_fewshot"
prompt_name = "3shots_with200to300words_withtag"

write_path = f'{proj_path}{model_name}/{method}/{data_type}/{prompt_name}/'

In [ ]:
print(write_path)

In [ ]:
def loader(path, sections):
    with open(path, 'r') as f:
        df = pd.read_json(path,
                    orient="records",
                    lines=True)
        return df[sections]

In [ ]:
sections = ['lay_summary', 'abstract', 'introduction', 'keywords']
path = f'{proj_path}data/{data_set}_train_mini.jsonl'
train_df = loader(path, sections)


sections = ['abstract', 'introduction', 'keywords']
path = f'{proj_path}data/{data_set}_{data_type}.jsonl'
input_df = loader(path, sections)

In [ ]:
train_df.iloc[0]

In [ ]:
input_df.iloc[0]

In [ ]:
len(input_df)

## Model

In [ ]:
sampling_params = SamplingParams(temperature=0.8, top_p=0.05, max_tokens=1024)

In [ ]:
import random
random.seed(531)

number_of_shot = 3
shot_idx = random.sample(train_df.index.tolist(), number_of_shot)
shot_df = train_df.iloc[shot_idx]

In [ ]:
prompt = ""
for idx in shot_idx:
    text = f"<s>[INST] Write a lay summary with 200 to 300 words, put the summary in 1 paragraph. Abstract: '{train_df.iloc[idx].abstract}', Introduction: '{train_df.iloc[idx].introduction}', Keyword: '{train_df.iloc[idx].keywords}' [/INST] Lay Summary: '{train_df.iloc[idx].lay_summary}'</s> "
    prompt += text

prompt += f"<s>[INST] Write a lay summary with 200 to 300 words, put the summary in 1 paragraph. Abstract: '{input_df.iloc[0].abstract}', Introduction: '{input_df.iloc[0].introduction}', Keyword: '{input_df.iloc[0].keywords}'[/INST] Lay Summary: "

In [ ]:
#prompt

In [ ]:
output = llm.generate(prompt, sampling_params)
answer = output[0].outputs[0].text.strip('\n').replace('\n', '')+'\n'

In [ ]:
len(prompt.split())

In [ ]:
def generate_prompt(train_df, input_df, number_of_shot):
    prompts = []
    for i in range(len(input_df)):
        shot_idx = random.sample(train_df.index.tolist(), number_of_shot)
        shot_df = train_df.iloc[shot_idx]
        prompt = ""
        for idx in shot_idx:
            text = f"<s>[INST] Write a lay summary with 200 to 300 words, put the summary in 1 paragraph. Abstract: '{train_df.iloc[idx].abstract}', Introduction: '{train_df.iloc[idx].introduction}', Keyword: '{train_df.iloc[idx].keywords}' [/INST] Lay Summary: '{train_df.iloc[idx].lay_summary}'</s> "
            prompt += text

        prompt += f"<s>[INST] Write a lay summary with 200 to 300 words, put the summary in 1 paragraph. Abstract: '{input_df.iloc[i].abstract}', Introduction: '{input_df.iloc[i].introduction}', Keyword: '{input_df.iloc[i].keywords}'[/INST] Lay Summary: "
        prompts.append(prompt)
    return prompts

In [ ]:
prompts = generate_prompt(train_df, input_df, number_of_shot)

In [ ]:
len(prompts)

In [ ]:
try:
    del output
except:
    print("First run in the day")
output = llm.generate(prompts, sampling_params)

In [ ]:
print(write_path + data_set.lower() + ".txt")

In [ ]:
with open(write_path + data_set.lower() + ".txt", 'w') as f:
    for item in output:
        f.write(item.outputs[0].text.strip('\n').replace('\n', '')+'\n')

In [ ]:
# from google.colab import runtime
# runtime.unassign()